<a href="https://colab.research.google.com/github/FrancescoZanella/Disjoint_set_maze/blob/main/Disjoint_set_maze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random Maze implementation using disjoint-set data-structure**



# **Notebook setup**


**Download the code to use the benchmark**


In [1]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 8228, done.
remote: Counting objects: 100% (1702/1702), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 8228 (delta 1548), reused 1502 (delta 1453), pack-reused 6526
Receiving objects: 100% (8228/8228), 2.56 MiB | 5.46 MiB/s, done.
Resolving deltas: 100% (5539/5539), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 26986, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 26986 (delta 7), reused 16 (delta 7), pack-reused 26961
Receiving objects: 100% (26986/26986), 12.39 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (20041/20041), done.


**Organize the code and install**

In [2]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- git version: v1.8.3-3-gca8d0f7b normalized to 1.8.3.3
-- Google Benchmark version: 1.8.3.3
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL - Success
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST - Success
-- Performing T

# **Implementation**

**maze.h**

In [3]:
%%writefile maze.h
#if!defined MAZE_H
#define MAZE_H

#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

typedef struct {
    int row;
    int col;
}cord;

struct disjoint_set {
    cord value;
    int height;
    struct disjoint_set* parent;
};
typedef struct disjoint_set disjoint_set;

void visualizemaze(bool* walls, int nrow, int ncols);
void union_set(disjoint_set* s1, disjoint_set* s2);
void find_the_cells_divided(cord* res1, cord* res2, int wall, int nrow, int ncol);
cord find_rep(disjoint_set* cell);
bool find(disjoint_set* s, cord values);
bool compare_cord(cord s, cord s1);
cord make_cord(int r, int c);
void make_set(disjoint_set* s, cord value);
bool* run_maze(int rows,int cols);
#endif /*MAZE_H*/


Writing maze.h


**maze.c**

In [4]:
%%writefile maze.c


#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <time.h>
#include"maze.h"

cord make_cord(int r, int c) {
    cord ret = { r,c };
    return ret;
}

bool compare_cord(cord s, cord s1) {
    if (s.row == s1.row && s.col == s1.col) { return true; }
    else {
        return false;
    }
}

void make_set(disjoint_set* s, cord value) {
    s->height = 1;
    s->value = value;
    s->parent = s;
}

//given a member of the set(cell) return the representative(set) to which it belongs
// follow the chain of parent pointers until to
//the root. O(height of x’s tree)
cord find_rep(disjoint_set* cell) {
    while (cell->parent != cell) {
        cell = cell->parent;
    }
    return cell->value;
}

// given a wall put the two cord of th cell in res
void find_the_cells_divided(cord* res1, cord* res2, int wall, int nrow, int ncol) {
    // Determine the row and the column
    int row = wall / (2 * ncol - 1);
    int column = wall % (2 * ncol - 1);
    if (column < ncol - 1) {
        res1->row = (ncol * row + column) / ncol;
        res1->col = (ncol * row + column) % ncol;
        res2->row = (ncol * row + column + 1) / ncol;
        res2->col = (ncol * row + column + 1) % ncol;
    }
    else {
        column += 1 - ncol;
        res1->row = (ncol * row + column) / ncol;
        res1->col = (ncol * row + column) % ncol;
        res2->row = (ncol * (row + 1) + column) / ncol;
        res2->col = (ncol * (row + 1) + column) % ncol;

    }

}

/*union of sets
given two sets, let the root of one tree point to the root of
 the other. O(1)
Each node is associated with a rank,
which is the upper bound on the height of the node, then
when UNION, let the root with smaller rank point to
the root with larger rank.*/

void union_set(disjoint_set* s1, disjoint_set* s2) {
    if (s1->height <= s2->height) {
        //allora appendi alla radice del maggiore l'albero minore
        s1->parent = s2;
        s2->height++;
    }
    else {
        s2->parent = s1;
        s1->height++;
    }
    return;
}






void visualizemaze(bool* walls, int nrow, int ncols)
{

    //riga iniziale
    printf("╔");
    for (int i = 0; i < ncols - 1; i++) {
        printf("════╦");
    }
    printf("════╗");
    printf("\n");
    //corpo
    for (int i = 0; i < nrow * 2 - 1; i++) {
        if (i % 2 == 0) {
            printf("║");
            for (int j = i / 2 * (ncols - 1) + i / 2 * ncols; j < i / 2 * (ncols - 1) + i / 2 * ncols + ncols - 1; j++) {
                if (walls[j]) {
                  if(i==0 && j ==0){
                        printf("  s ║");
                  }
                  else{
                    if(i==nrow*2 && j==ncols-1){
                        printf("  g ║");
                    }
                    else{
                        printf("    ║");
                    }

                  }
                }
                else {
                  if(i==0 && j ==0){
                    printf("  s  ");
                  }
                  else{
                    printf("     ");
                      }
                }
            }
            if(i==nrow*2-2 /*&& j==ncols-1*/){
                   printf(" g  ║\n");
            }
            else{
              printf("    ║\n");
            }

        }
        else {
            printf("╠");
            for (int j = (i / 2) * ncols + (i / 2 + 1) * (ncols - 1); j < (i / 2) * ncols + (i / 2 + 1) * (ncols - 1) + ncols; j++) {
                if (walls[j]) {
                    if (j != (i / 2) * ncols + (i / 2 + 1) * (ncols - 1)) {
                        printf("╬════");
                    }
                    else {
                        printf("════");
                    }
                }
                else {
                    if (j != (i / 2) * ncols + (i / 2 + 1) * (ncols - 1)) {
                        printf("╬    ");
                    }
                    else {
                        printf("    ");
                    }
                }
            }
            printf("╣\n");

        }
    }

    printf("╚");
    for (int i = 0; i < ncols - 1; i++) {
        printf("════╩");
    }
    printf("════╝");

}


bool* run_maze(int rows,int cols){
  int nwalls = rows * cols * 2 - rows - cols;

    //initialize the seed of the random number
    srand(time(NULL));
    //there are rows*cols sets
    //allocate the matrix of sets
    disjoint_set** cells = (disjoint_set**)malloc(rows * sizeof(disjoint_set*));
    for (int i = 0; i < rows; i++) {
        cells[i] = (disjoint_set*)malloc(cols * sizeof(disjoint_set));
    }

    //initialize the rows * cols disjoint_set
    for (int r = 0; r < rows; r++) {
        for (int c = 0; c < cols; c++) {
            make_set(&cells[r][c], make_cord(r, c));
        }
    }

    //allocate the walls and set alls to true
    bool* walls = (bool*)malloc(nwalls * sizeof(bool));
    for (int i = 0; i < nwalls; i++) {
        walls[i] = true;
    }


    //until the first start cell and the goal cell are not in the same set continue to delete walls
    while (!compare_cord(find_rep(&cells[0][0]), find_rep(&cells[rows - 1][cols - 1]))) {
        //pick a random wall among the nwalls available
        int random = rand() % nwalls;
        cord res1 = { 0,0 };
        cord res2 = { 0,0 };
        //given a wall return the cordinate(r,c) of the two cells that divide
        find_the_cells_divided(&res1, &res2, random, rows, cols);
        //if and only if the two cells are not in the same set delete the wall and make a union
        if (!compare_cord(find_rep(&cells[res1.row][res1.col]), find_rep(&cells[res2.row][res2.col]))) {
            //delete the wall
            walls[random] = false;
            union_set(&cells[find_rep(&cells[res1.row][res1.col]).row][find_rep(&cells[res1.row][res1.col]).col], &cells[find_rep(&cells[res2.row][res2.col]).row][find_rep(&cells[res2.row][res2.col]).col]);
        }
    }
    //free the memory previously allocated
    for(int i =0;i<rows;i++){
      free(cells[i]);
    }
    free(cells);
    return walls;
}



Writing maze.c


**main.c**

In [5]:
%%writefile main.c

#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <time.h>
#include"maze.h"



int main(int argc,char** argv)
{
    int rows = atoi(argv[1]);
    int cols = atoi(argv[2]);
    bool* walls = run_maze(rows,cols);

    //visualize the maze in a graphic way
    visualizemaze(walls, rows, cols);

    return 0;
}


Writing main.c


# **Compile the code**

In [6]:
!gcc main.c maze.c

Usage:
pass the number of rows and the columns desired as arguments.

!./a.out rows cols

# **Output**

In [8]:
!./a.out 10 10

╔════╦════╦════╦════╦════╦════╦════╦════╦════╦════╗
║  s      ║    ║    ║         ║                   ║
╠════╬    ╬    ╬    ╬════╬    ╬════╬    ╬════╬    ╣
║    ║              ║    ║         ║         ║    ║
╠    ╬    ╬════╬════╬    ╬════╬    ╬════╬    ╬    ╣
║         ║    ║              ║              ║    ║
╠    ╬════╬    ╬════╬════╬    ╬    ╬════╬    ╬════╣
║              ║    ║                   ║         ║
╠════╬════╬    ╬    ╬════╬════╬    ╬════╬════╬════╣
║         ║         ║              ║              ║
╠    ╬════╬    ╬════╬════╬    ╬    ╬════╬    ╬════╣
║              ║         ║    ║         ║         ║
╠════╬════╬    ╬    ╬════╬    ╬════╬════╬════╬    ╣
║    ║    ║    ║    ║              ║              ║
╠    ╬    ╬    ╬    ╬════╬════╬    ╬    ╬════╬════╣
║         ║                   ║                   ║
╠    ╬════╬    ╬════╬    ╬════╬════╬    ╬    ╬════╣
║                   ║                   ║         ║
╠════╬    ╬════╬════╬════╬════╬    ╬    ╬    ╬    ╣
║           

# **Profiling the algorithm**

In [9]:
%%writefile bench.cpp

#include <benchmark/benchmark.h>

extern "C" {
  #include"maze.c"
}
static void BM_RandomMaze(benchmark::State& state) {
  // Perform setup here
  int N = state.range(0);
  for (auto _ : state) {
    // This code gets timed
    run_maze(N,N);

  }
  state.SetComplexityN(N);
}
// Register the function as a benchmark
BENCHMARK(BM_RandomMaze)
  ->RangeMultiplier(2)
  ->Range(2,1024)
  ->Complexity();
// Run the benchmark
BENCHMARK_MAIN();

Writing bench.cpp


In [10]:
!g++ bench.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o testprogram1

# **Discuss the complexity**

Avendo implementato il disjoint set con la forest:
1.   MAKE_SET() complessità O(1)
2.   FIND_SET() ha complessità O(the height of the tree) dato che non è stata
implementata la path compression, ogni volta per trovare il valore rappresentativo di un set bisogna scorrere tutti i nodi(celle) fino alla radice, se invece avessi implementato la path compression ogni nodo era direttamente connesso alla radice e quindi avremmo avuto O(1) come complessita anche per la find.
3.   UNION_SET(): per la union set è stata implementata la union_by_rank di conseguenza ogni union avrà complessità O(1).

COMPLESSIVAMENTE LA MIA IMPLEMENTAZIONE HA COMPLESSITA' COMPUTAZIONE(come anche mostrato nel benchmark sotto):


*   worst case O(n^3)
*   average case O(n^2)
*   best case poco più che O(1)


In [11]:
!./testprogram1

2023-10-13T07:55:12+00:00
Running ./testprogram1
Run on (2 X 2200.19 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.72, 0.67, 0.33
-------------------------------------------------------------
Benchmark                   Time             CPU   Iterations
-------------------------------------------------------------
BM_RandomMaze/2          2012 ns         1961 ns       356555
BM_RandomMaze/4          2777 ns         2727 ns       284172
BM_RandomMaze/8          8236 ns         7998 ns        86700
BM_RandomMaze/16        36784 ns        36174 ns        16362
BM_RandomMaze/32       218904 ns       212131 ns         3933
BM_RandomMaze/64      1005837 ns       967835 ns          739
BM_RandomMaze/128    11096448 ns     10722480 ns           67
BM_RandomMaze/256    11181786 ns     10881063 ns           48
BM_RandomMaze/512    76778003 ns     75406870 ns            9
BM_RandomMaze/1024 100045968